In [5]:
from backend.db.models import IndicatorDimension, IndicatorPeriod, IndicatorRecord

from sqlalchemy import desc, func, select

Print query to select top contents

In [6]:
start_ts = 12
stop_ts = 13

subquery = (
            select(
                IndicatorDimension.value0.label("content"),
                func.sum(IndicatorRecord.value).label("count"),
            )
            .join(IndicatorRecord)
            .join(IndicatorPeriod)
            .where(IndicatorPeriod.timestamp >= start_ts)
            .where(IndicatorPeriod.timestamp <= stop_ts)
            .group_by("content")
        ).subquery("content_with_count")

query = select(subquery.c.content).order_by(desc(subquery.c.count)).limit(50)
print(query)


SELECT content_with_count.content 
FROM (SELECT indicator_dimension.value0 AS content, sum(indicator_record.value) AS count 
FROM indicator_dimension JOIN indicator_record ON indicator_dimension.id = indicator_record.dimension_id JOIN indicator_period ON indicator_period.id = indicator_record.period_id 
WHERE indicator_period.timestamp >= :timestamp_1 AND indicator_period.timestamp <= :timestamp_2 GROUP BY content) AS content_with_count ORDER BY content_with_count.count DESC
 LIMIT :param_1
